In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle

In [2]:
def load_data(path):
    with open(path) as fil:
        data = pd.read_csv(fil,sep='\t',header=None,names=["hi","en",""],skip_blank_lines=True,index_col=None)
    data = data[data['hi'].notna()]
    data = data[data['en'].notna()]
    data = data[['hi','en']]
    return data

In [3]:
train = load_data("hi.translit.sampled.train.tsv")
dev = load_data("hi.translit.sampled.dev.tsv")
test = load_data("hi.translit.sampled.test.tsv")

In [4]:
train.head()

,hi,en
0,अं,an
1,अंकगणित,ankganit
2,अंकल,uncle
3,अंकुर,ankur
4,अंकुरण,ankuran


In [5]:
x = train['en'].values
y = train['hi'].values
y = '\t'+y+'\n'

In [6]:
english_tokens = set()
hindi_tokens = set()

for xx,yy in zip(x,y):
    for ch in xx:
        english_tokens.add(ch)
    for ch in yy:
        hindi_tokens.add(ch)
    
english_tokens = sorted(list(english_tokens))
hindi_tokens = sorted(list(hindi_tokens))

In [7]:
eng_token_map = dict([(ch,i+1) for i,ch in enumerate(english_tokens)])
hin_token_map = dict([(ch,i+1) for i,ch in enumerate(hindi_tokens)])

In [8]:
hin_token_map[" "] = 0
eng_token_map[" "] = 0

In [9]:
max_eng_len = max([len(i) for i in x])
max_hin_len = max([len(i) for i in y])

In [10]:
def process(data):
    x,y = data['en'].values, data['hi'].values
    y = "\t" + y + "\n"
    
    a = np.zeros((len(x),max_eng_len),dtype="float32")
    b = np.zeros((len(y),max_hin_len),dtype="float32")
    c = np.zeros((len(y),max_hin_len,len(hindi_tokens)+1),dtype="int")
    
    
    for i,(xx,yy) in enumerate(zip(x,y)):
        for j,ch in enumerate(xx):
            a[i,j] = eng_token_map[ch]

        a[i,j+1:] = eng_token_map[" "]
        for j,ch in enumerate(yy):
            b[i,j] = hin_token_map[ch]

            if j>0:
                c[i,j-1,hin_token_map[ch]] = 1

        b[i,j+1:] = hin_token_map[" "]
        c[i,j:,hin_token_map[" "]] = 1
        
    return a,b,c

In [11]:
trainx, trainxx, trainy = process(train)
valx, valxx, valy = process(dev)
testx,testxx,testy = process(test)

In [12]:
np.random.seed(42)

In [13]:
reverse_eng_map = dict([(i,char) for char,i in eng_token_map.items()])
reverse_hin_map = dict([(i,char) for char,i in hin_token_map.items()])

## Training

In [14]:
import keras
import tensorflow as tf
from keras.layers import SimpleRNN,LSTM,GRU,Embedding,Dense,Dropout,Input,Concatenate
from keras.optimizers import Adam,Nadam
from keras import Model

In [15]:
class Attention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(Attention, self).__init__()
        self.W1 = Dense(units)
        self.W2 = Dense(units)
        self.V = Dense(1)

    def call(self, query, values):
        # query hidden state shape == (batch_size, hidden size)
        # query_with_time_axis shape == (batch_size, 1, hidden size)
        # values shape == (batch_size, max_len, hidden size)
        # we are doing this to broadcast addition along the time axis to calculate the score
        query_with_time_axis = tf.expand_dims(query, 1)
        # score shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying score to self.V
        # the shape of the tensor before applying self.V is (batch_size, max_length, units)
        score = self.V(tf.nn.tanh(
            self.W1(query_with_time_axis) + self.W2(values)))

        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)

        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)
        

        return context_vector, attention_weights

In [16]:
def accuracy1(real,pred):
    real = tf.math.argmax(real,axis=2)
    pred = tf.math.argmax(pred,axis=2)
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    acc = tf.math.equal(real,pred)
    mask = tf.cast(mask, dtype='int32')
    acc = tf.cast(acc, dtype='int32')
    acc = tf.math.multiply(acc,mask)
    mask = tf.reduce_sum(mask,axis=1)
    acc = tf.reduce_sum(acc,axis=1)
    acc = tf.math.equal(acc,mask)
    acc = tf.cast(acc, dtype='float32')
    return tf.reduce_mean(acc)

In [17]:
def build_model(cell = "LSTM",nunits = 32, enc_layers = 1, dec_layers = 1,embed_dim = 32,dense_size=32,dropout=None):
    keras.backend.clear_session()
    encoder_inputs = Input(shape=(None,))
    encoder_embedding = Embedding(input_dim=len(english_tokens)+1,output_dim = embed_dim,mask_zero=True)
    encoder_context = encoder_embedding(encoder_inputs)
    decoder_inputs = Input(shape=(None,))
    decoder_embedding = Embedding(input_dim = len(hindi_tokens)+1,output_dim = embed_dim,mask_zero=True)
    decoder_context = decoder_embedding(decoder_inputs)
    attention = Attention(nunits)
    tot_out = []
    concat1 = Concatenate(axis=-1)
    concat2 = Concatenate(axis=1)    
    if cell == "LSTM":
        encoder_prev = [LSTM(nunits,return_sequences=True) for i in range(enc_layers-1)]
        encoder_fin = LSTM(nunits,return_sequences=True,return_state=True)
        temp = encoder_context
        for lay in encoder_prev:
            temp = lay(temp)
            if dropout is not None:
                temp = Dropout(dropout)(temp)
            
        enc_out = encoder_fin(temp)
        dec_states = enc_out[1:]
        
        decoder = [LSTM(nunits,return_sequences=True,return_state=True) for i in range(dec_layers)]
        temp_states = [dec_states]*dec_layers
        
        for i in range(max_hin_len):
            context,att_wts = attention(temp_states[0][0],enc_out[0])
            temp = concat1([tf.expand_dims(context, 1), decoder_context[:,i:i+1,:]])
#             temp = tf.expand_dims(temp, 1)
            for i in range(dec_layers):
                temp,sh,sc = decoder[i](temp,initial_state=temp_states[i])
                temp_states[i] = [sh,sc]
            tot_out.append(temp)
            
        outt = concat2(tot_out)
       
    elif cell == "GRU":
        encoder_prev = [GRU(nunits,return_sequences=True) for i in range(enc_layers-1)]
        encoder_fin = GRU(nunits,return_sequences=True,return_state=True)
        temp = encoder_context
        for lay in encoder_prev:
            temp = lay(temp)
            if dropout is not None:
                temp = Dropout(dropout)(temp)
            
        enc_out = encoder_fin(temp)
        dec_states = enc_out[1:]
        
        decoder = [GRU(nunits,return_sequences=True,return_state=True) for i in range(dec_layers)]
        temp_states = []
        for _ in range(dec_layers):
            temp_states += dec_states
        
        for i in range(max_hin_len):
            context,att_wts = attention(temp_states[0],enc_out[0])
            temp = concat1([tf.expand_dims(context, 1), decoder_context[:,i:i+1,:]])
#             temp = tf.expand_dims(temp, 1)
            for i in range(dec_layers):
                temp,st = decoder[i](temp,initial_state=temp_states[i])
                temp_states[i] = st
            tot_out.append(temp)
            
        outt = concat2(tot_out)
            
        
    dense_lay1 = Dense(dense_size,activation='relu')
    pre_out = dense_lay1(outt)
    dense_lay2 = Dense(len(hindi_tokens)+1,activation = 'softmax')
    final_output = dense_lay2(pre_out)
    
    train = Model([encoder_inputs,decoder_inputs],final_output)
    
    encoder_model = Model(encoder_inputs,enc_out)
    
    if cell == "LSTM":
        state_inputs = []
        state_outputs = []
        
        encout_input = Input(shape=(None,nunits))
        
        temp = decoder_context
                                                                  
        for i in range(dec_layers):
            decoder_input_h = Input(shape=(nunits,))
            decoder_input_c = Input(shape=(nunits,))
            
            if i==0:
                context,att_wts_out = attention(decoder_input_h,encout_input)
                temp = concat1([tf.expand_dims(context, 1), temp])
                
            temp,sh,sc = decoder[i](temp,initial_state = [decoder_input_h,decoder_input_c])
            state_inputs += [decoder_input_h,decoder_input_c]
            state_outputs += [sh,sc]
            
        decoder_input_pass = [decoder_inputs,encout_input] + state_inputs
        
    elif cell == "GRU":
        state_inputs = []
        state_outputs = []
        
        encout_input = Input(shape=(None,nunits))
        
        temp = decoder_context
                                                                  
        for i in range(dec_layers):
            state_input = Input(shape=(nunits,))
            
            if i==0:
                context,att_wts_out = attention(state_input,encout_input)
                temp = concat1([tf.expand_dims(context, 1), temp])
                
            temp,s = decoder[i](temp,initial_state = state_input)
            state_inputs.append(state_input)
            state_outputs.append(s)
            
        decoder_input_pass = [decoder_inputs,encout_input] + state_inputs

    pre_out = dense_lay1(temp)
    final_output = dense_lay2(pre_out)
    
    decoder_model = Model(decoder_input_pass, [final_output,att_wts_out]+state_outputs)
    
    return train,encoder_model,decoder_model
#     return train

In [19]:
train,enc,dec = build_model(nunits=256,
                                dense_size=128,
                                enc_layers=1,
                                dec_layers=1,
                                cell = "GRU",
                                dropout = 0.0,
                                embed_dim = 128)
train.compile(optimizer = Adam(),loss='categorical_crossentropy',metrics=[accuracy1])

In [21]:
train.fit([trainx,trainxx],trainy,
             batch_size=64,
             validation_data = ([valx,valxx],valy),
             epochs=7)

Epoch 1/7
691/691 [==============================] - 47s 69ms/step - loss: 0.0900 - accuracy1: 0.5288 - val_loss: 0.1447 - val_accuracy1: 0.3880
Epoch 2/7
691/691 [==============================] - 47s 69ms/step - loss: 0.0809 - accuracy1: 0.5621 - val_loss: 0.1483 - val_accuracy1: 0.3810
Epoch 3/7
691/691 [==============================] - 48s 69ms/step - loss: 0.0737 - accuracy1: 0.5885 - val_loss: 0.1482 - val_accuracy1: 0.3879
Epoch 4/7
691/691 [==============================] - 48s 69ms/step - loss: 0.0644 - accuracy1: 0.6271 - val_loss: 0.1504 - val_accuracy1: 0.3862
Epoch 5/7
691/691 [==============================] - 47s 69ms/step - loss: 0.0582 - accuracy1: 0.6542 - val_loss: 0.1596 - val_accuracy1: 0.3898
Epoch 6/7
691/691 [==============================] - 47s 69ms/step - loss: 0.0533 - accuracy1: 0.6763 - val_loss: 0.1641 - val_accuracy1: 0.3887
Epoch 7/7
691/691 [==============================] - 47s 69ms/step - loss: 0.0476 - accuracy1: 0.7011 - val_loss: 0.1630 - val_acc

## WandB Sweep

In [23]:
import wandb
from wandb.keras import WandbCallback

In [29]:
sweep_config = {
    'method': 'random', #grid, random
    'metric': {
      'name': 'val_accuracy1',
      'goal': 'maximize'   
    },
    'parameters': {
        'lr': {
            'values': [1e-2,1e-3]
        },
        'dense_size': {
            'values': [64,128,512]
        },
        'dropout': {
            'values': [0.0,0.2]
        },
        'nunits': {
            'values': [128,256,512]
        },
        'cell': {
            'values': ["LSTM","GRU"]
        },
        'embed_size': {
            'values': [64,128,256]
        },
        'enc_layers': {
            'values': [1,2,3]
        },
        'dec_layers': {
            'values': [1,2,3]
        },
        
    }
}

In [24]:
sweep_id = wandb.sweep(sweep_config,entity = "kris21" , project = "cs6910-a3")

Create sweep with ID: xd9jptuj
Sweep URL: https://wandb.ai/kris21/cs6910-a3/sweeps/xd9jptuj


In [32]:
def train():
    # Default values for hyper-parameters we're going to sweep over
    config_defaults = {
        'lr': 1e-2,
        'dense_size': 128,
        'nunits': 128,
        'cell': 'LSTM',
        'embed_size': 64,
        'enc_layers': 1,
        'dec_layers': 1,
        'dropout': 0.
    }

    # Initialize a new wandb run
    wandb.init(config=config_defaults,name="cs6910-a2")
    
    cfg = wandb.config
    
    name = f'nunits_{cfg.nunits}_cell_{cfg.cell}_encl_{cfg.enc_layers}_decl_{cfg.dec_layers}_emb_{cfg.embed_size}_ds_{cfg.dense_size}_do_{cfg.dropout}_lr_{cfg.lr}'
    wandb.run.name = name
    wandb.run.save()
    
    # Config is a variable that holds and saves hyperparameters and inputs
    
    train,enc,dec = build_model(nunits=cfg.nunits,
                                dense_size=cfg.dense_size,
                                enc_layers=cfg.enc_layers,
                                dec_layers=cfg.dec_layers,
                                cell = cfg.cell,
                                dropout = cfg.dropout,
                                embed_dim = cfg.embed_size)
    train.compile(optimizer = Adam(lr=cfg.lr),loss='categorical_crossentropy',metrics=[accuracy1])
    print("model building done")
    train.fit([trainx,trainxx],trainy,
             batch_size=64,
             validation_data = ([valx,valxx],valy),
             epochs=7,
             callbacks = [WandbCallback(monitor='val_accuracy1',mode='max')])
    print("model training done")


In [33]:
wandb.agent(sweep_id, train,count=20)

wandb: Agent Starting Run: h3r5nt5m with config:
wandb: 	cell: LSTM
wandb: 	dec_layers: 3
wandb: 	dense_size: 64
wandb: 	dropout: 0
wandb: 	embed_size: 64
wandb: 	enc_layers: 3
wandb: 	lr: 0.001
wandb: 	nunits: 128
wandb: wandb version 0.10.30 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


model building done
Epoch 1/7
691/691 [==============================] - 222s 172ms/step - loss: 1.3862 - accuracy1: 1.6038e-05 - val_loss: 0.8110 - val_accuracy1: 6.7935e-04


wandb: ERROR Can't save model, h5py returned error: Layer Attention has arguments in `__init__` and therefore must override `get_config`.


Epoch 2/7
691/691 [==============================] - 91s 132ms/step - loss: 0.7787 - accuracy1: 0.0014 - val_loss: 0.5878 - val_accuracy1: 0.0061
Epoch 3/7
691/691 [==============================] - 92s 133ms/step - loss: 0.5695 - accuracy1: 0.0171 - val_loss: 0.4325 - val_accuracy1: 0.0500
Epoch 4/7
691/691 [==============================] - 90s 130ms/step - loss: 0.4118 - accuracy1: 0.0806 - val_loss: 0.3238 - val_accuracy1: 0.1372
Epoch 5/7
691/691 [==============================] - 90s 130ms/step - loss: 0.3087 - accuracy1: 0.1714 - val_loss: 0.2687 - val_accuracy1: 0.1983
Epoch 6/7
691/691 [==============================] - 90s 130ms/step - loss: 0.2492 - accuracy1: 0.2402 - val_loss: 0.2345 - val_accuracy1: 0.2355
Epoch 7/7
691/691 [==============================] - 91s 132ms/step - loss: 0.2112 - accuracy1: 0.2867 - val_loss: 0.2158 - val_accuracy1: 0.2731
model training done


...

model building done
Epoch 1/7
691/691 [==============================] - 155s 125ms/step - loss: 1.2114 - accuracy1: 0.0022 - val_loss: 0.3826 - val_accuracy1: 0.0906


wandb: ERROR Can't save model, h5py returned error: Layer Attention has arguments in `__init__` and therefore must override `get_config`.


Epoch 2/7
691/691 [==============================] - 69s 99ms/step - loss: 0.3075 - accuracy1: 0.1734 - val_loss: 0.1912 - val_accuracy1: 0.2926
Epoch 3/7
691/691 [==============================] - 68s 99ms/step - loss: 0.1584 - accuracy1: 0.3644 - val_loss: 0.1634 - val_accuracy1: 0.3369
Epoch 4/7
691/691 [==============================] - 69s 99ms/step - loss: 0.1137 - accuracy1: 0.4840 - val_loss: 0.1520 - val_accuracy1: 0.3724
Epoch 5/7
691/691 [==============================] - 69s 99ms/step - loss: 0.0823 - accuracy1: 0.5832 - val_loss: 0.1557 - val_accuracy1: 0.3678
Epoch 6/7
691/691 [==============================] - 68s 98ms/step - loss: 0.0620 - accuracy1: 0.6573 - val_loss: 0.1558 - val_accuracy1: 0.3714
Epoch 7/7
691/691 [==============================] - 69s 99ms/step - loss: 0.0463 - accuracy1: 0.7278 - val_loss: 0.1695 - val_accuracy1: 0.3721
model training done


epoch,6
loss,0.05012
accuracy1,0.7062
val_loss,0.16951
val_accuracy1,0.37206
_runtime,588
_timestamp,1620850423
_step,6
best_val_accuracy1,0.37243
best_epoch,3


epoch,▁▂▃▅▆▇█
loss,█▃▂▂▁▁▁
accuracy1,▁▃▅▆▇▇█
val_loss,█▂▁▁▁▁▂
val_accuracy1,▁▆▇████
_runtime,▁▂▃▅▆▇█
_timestamp,▁▂▃▅▆▇█
_step,▁▂▃▅▆▇█
